## 뉴스 크롤링 후 저장 

In [31]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, json, glob, time
from tqdm import tqdm



###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5


####### 언론사별 본문 위치 태그 파싱 함수 ###########
print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('article', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 그 외
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
press_nm = 'MBC'

print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = 'C:/Users/jhjh3/chromedriver_win32 (1)/chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)


######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:
    
    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)
    
    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)
        
        break



################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)
    
while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}
        
        idx += 1
        pbar.update(1)
        
    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()
        
        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break

#### 데이터 전처리하기 ###################################################### 

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : MBC
검색할 키워드  : 코로나
수집 뉴스의 수(숫자만 입력) : 3


브라우저를 실행시킵니다(자동 제어)



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20648/1155766905.py:83: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20648/1155766905.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20648/1155766905.py:97: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')


<방송/통신> 카테고리에서 <MBC>를 찾았으므로 탐색을 종료합니다

크롤링을 시작합니다.


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20648/1155766905.py:152: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_xpath('//ul[@class="list_news"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.06s/it]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\jhjh3\PJT_02\네이버뉴스_본문_3개_코로나_2022-03-22_16시19분.xlsx

결과물의 일부


,title,url,text
1,"코로나19 신규 확진 35만3,980명‥사망 384명",https://imnews.imbc.com/news/2022/society/arti...,사진 제공: 연합뉴스중앙방역대책본부는 오늘 0시 기준 코로나19 신규 확진자가 35...
2,"""코로나19 사태 속 저소득층 물가부담, 고소득층 1.4배""",https://imnews.imbc.com/news/2022/econo/articl...,사진 제공: 연합뉴스코로나19 사태 속에서 물가 상승에 따른 부담이상대적으로 저소득...
3,"축구대표팀 소집된 백승호, 코로나19 확진‥원두재 대체 발탁",https://imnews.imbc.com/news/2022/sports/artic...,사진 제공:연합뉴스이란과의 월드컵 최종예선을 앞둔 축구대표팀에 코로나19 비상이 걸...


In [15]:
news_df.text[1]

'김잔디 기자기자 페이지식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 \'라게브리오\'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했다. 식약처는 "긴급사용승인 여부는 식약처의 검토 이후에 공중보건 위기대응 의료제품위원회 심의를 거쳐 식약처장이 결정하는 사안"이라며 "엄중한 상황임을 고려해 신속히 위원회를 개최하고 늦어도 이번 주 목요일(24일)까지 라게브리오의 긴급사용승인 여부를 발표하겠다"고 21일 밝혔다.  이어 식약처는 "비임상시험에서 제기된 안전성 우려와 관련해 임부나 소아의 사용을 제한하는 방향으로 검토 중"이라고 덧붙였다. 광고 긴급사용승인은 감염병 대유행 등 공중보건 위기 상황에 적절히 대응하기 위해 제조·수입자가 국내에 허가되지 않은 의료제품을 공급하는 제도다. 질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했다.  현재 식약처는 코로나19 환자가 폭증함에 따라 \'팍스로비드\'나 \'렘데시비르\'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중이다. 안전성 및 유효성에 대한 자료, 외국의 허가·승인 및 사용 현황, 대체치료제 현황 등을 종합적으로 살펴보고 있다. 라게브리오는 지난해 12월 미국 식품의약국(FDA)의 긴급사용승인을 획득했다. FDA는 고위험군의 경증 코로나19 환자 중에서 다른 치료제가 없거나 의학적으로 적합한 경우에 몰누피라비르(라게브리오의 성분명)를 쓸 수 있도록 허용했다. 단 18세 이하 환자는 성장에 영향을 미칠 수 있다는 이유로 사용을 금지했다. 가임기 여성 환자는 약 복용 후 일정 기간 피임해야 하고, 남성은 마지막 복용 후 세달은 피임해야 한다고 봤다. 

In [14]:
response.text 

'{"summary":"김잔디 기자기자 페이지식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토\\"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중\\" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 \'라게브리오\'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했습니다.\\n질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했습니다.\\n현재 식약처는 코로나19 환자가 폭증함에 따라 \'팍스로비드\'나 \'렘데시비르\'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중입니다."}'

In [19]:
import requests
import json

url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"

document = {"title": news_df.title[1],
            "content": news_df.text[1]}
            
option =  {"language": "ko", 
           "model": "news",
           "tone": 2,
           "summaryCount": 3}

msg = {'document':document, 'option':option}

payload = json.dumps(msg)


headers = {
  'X-NCP-APIGW-API-KEY-ID': 'qj40wl5n0t',
  'X-NCP-APIGW-API-KEY': '1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

# print(response.text)

r1=response.text  
r1r1=r1.split('페이지')[1]
print(r1r1)

식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토\"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중\" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 '라게브리오'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했습니다.\n질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했습니다.\n현재 식약처는 코로나19 환자가 폭증함에 따라 '팍스로비드'나 '렘데시비르'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중입니다."}


In [19]:
url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"
document = {"title": news_df.title[1],
            "content": news_df.text[1]}            
option =  {"language": "ko", 
           "model": "news",
           "tone": 2,
           "summaryCount": 3}
msg = {'document':document, 'option':option}
payload = json.dumps(msg)
headers = {
  'X-NCP-APIGW-API-KEY-ID': '*',
  'X-NCP-APIGW-API-KEY': '*',
  'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

r1=response.text  
r1r1=r1.split('페이지')[1]
print(r1r1)

식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토\"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중\" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 '라게브리오'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했습니다.\n질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했습니다.\n현재 식약처는 코로나19 환자가 폭증함에 따라 '팍스로비드'나 '렘데시비르'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중입니다."}


In [25]:
len(news_df.title)

import os
import sys
import requests
import json
import urllib.request

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(1, len(news_df.title)+1):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text  
        # print(r1)

    else:
        print("Error : " + response.text)

    r1r1=r1.split('페이지')[1]
    print(r1r1)


식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토\"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중\" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 '라게브리오'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했습니다.\n질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했습니다.\n현재 식약처는 코로나19 환자가 폭증함에 따라 '팍스로비드'나 '렘데시비르'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중입니다."}
Error : {"status":400,"error":{"errorCode":"E003","message":"Text quota Exceeded"}}
식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토\"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중\" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 '라게브리오'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했습니다.\n질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했습니다.\n현재 식약처는 코로나19 환자가 폭증함에 따라 '팍스로비드'나 '렘데시비르'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중입니다."}
\"지표변동은 일단 긍정적…이런 경향 반복된다면 지난주 정점\"한의원서도 전문가용 신속항원검사 요청에 \"관련내용 검토중\"코로나19 신속항원검사 결과 대기(서울=연합뉴스) 강민지 기자 = 21일 오전 서울역 선별진료소에서 시민들이 신속항원검사 결과를 기다리고 있습니다.\

#### 요약 후 음성파일에 넣는데 뉴스 기사 구성마다 차이가 있어서 오류가 나는 뉴스도 확인된다...

In [26]:
len(news_df.title)

import os
import sys
import requests
import json
import urllib.request

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(1, len(news_df.title)+1):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text  
        # print(r1)

    else:
        print("Error : " + response.text)

    r1r1=r1.split('페이지')[1]
    print(r1r1)


    ## 읽어줘요

    encText = urllib.parse.quote(r1r1)
    data = "speaker=nara&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
    url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode('utf-8'))
    rescode = response.getcode()
    if(rescode==200):
        print("TTS mp3 저장")
        response_body = response.read()
        with open('1111.mp3', 'wb') as f:
            f.write(response_body)
    else:
        print("Error Code:" + rescode)

식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토\"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중\" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 '라게브리오'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했습니다.\n질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했습니다.\n현재 식약처는 코로나19 환자가 폭증함에 따라 '팍스로비드'나 '렘데시비르'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중입니다."}
TTS mp3 저장
Error : {"status":400,"error":{"errorCode":"E003","message":"Text quota Exceeded"}}
식약처, 지난해 11월 질병청 요청 후 긴급사용승인 여부 검토\"안전성 우려에 따라 임부나 소아 사용 제한하는 방향으로 검토 중\" (서울=연합뉴스) 김잔디 기자 = 정부가 미국 머크앤드컴퍼니(MSD)의 먹는 코로나19 치료제 '라게브리오'(성분명 몰누피라비르)를 이번 주에 도입한다고 공식화한 가운데 식품의약품안전처가 늦어도 목요일인 24일까지는 이 약의 긴급사용승인 여부를 결정키로 했습니다.\n질병관리청은 지난해 11월 17일 식약처에 라게브리오의 긴급사용승인을 요청했습니다.\n현재 식약처는 코로나19 환자가 폭증함에 따라 '팍스로비드'나 '렘데시비르'를 사용하기 어려운 고위험 경증∼중등증 환자들을 위한 추가 선택지로서 라게브리오의 긴급사용승인 필요성을 검토 중입니다."}
TTS mp3 저장
\"지표변동은 일단 긍정적…이런 경향 반복된다면 지난주 정점\"한의원서도 전문가용 신속항원검사 요청에 \"관련내용 검토중\"코로나19 신속항원검사 결과 대기(서울=연합뉴스) 강민지 기자 = 21일 오전 서울역 선별진료소에서 시민들이 

In [28]:
import os
import sys
import urllib.request
client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"
encText = urllib.parse.quote(news_list)
data = "speaker=njinho&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request, data=data.encode('utf-8'))
rescode = response.getcode()
if(rescode==200):
    print("TTS mp3 저장")
    response_body = response.read()
    with open('1115.mp3', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

TypeError: quote_from_bytes() expected bytes

## 뉴스 구분 넣고, 음성파일 voice{}.mp3로 각각 만들기

In [74]:
len(news_df.title)
# news_list = []

import os
import sys
import requests
import json
import urllib.request

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(len(news_df.title)):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i+1]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text  
        # print(r1)

    else:
        print("Error : " + response.text)

    r1 = json.loads(r1)
    r1r1= r1['summary']
#    r1r1= r1.split('연합뉴스')   
    r1r1 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)
    print(r1r1)
    

#print(news_list)
#type(news_list)

    ## 읽어줘요

    encText = urllib.parse.quote(r1r1)
    data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
    url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode('utf-8'))
    rescode = response.getcode()
    if(rescode==200):
        print("TTS mp3 저장")
        response_body = response.read()
        fn = 'voice{}.mp3'.format(i+1,fn)
        
        with open(fn, 'wb') as f:
            f.write(response_body)
    else:
        print("Error Code:" + rescode)
        
#playsound('voice1.mp3')
#playsound.playsound('voice1.mp3')


1번째 뉴스입니다.사진 제공: 연합뉴스중앙방역대책본부는 오늘 0시 기준 코로나19 신규 확진자가 35만3,980명으로 집계됐다고 밝혔습니다.
지역별로는 경기 96,257명, 서울 65,033명 등 수도권에서 17만8,859명이 확진됐고, 비수도권에서는 17만5,121명이 양성 판정을 받았습니다.
위중증 환자는 26명이 줄어 1천104명이고, 어제 하루 코로나로 인한 사망자는 384명으로 코로나19 사태 발생 이후 역대 두번째로 많았습니다.
TTS mp3 저장
2번째 뉴스입니다.사진 제공: 연합뉴스코로나19 사태 속에서 물가 상승에 따른 부담이상대적으로 저소득층에 집중됐다는 분석이 나왔습니다.
한경연은 이같은 결과에 대해 저소득층 지출 비중이 높은 품목의 물가가 상대적으로 크게 상승했기 때문이라고 설명했습니다.
저소득층인 소득 1분위 지출 비중이 5분위보다 높은 <주거·수도·광열>, <식료품>, <보건> 물가는 평균 3.7% 상승한 반면 5분위의 지출 비중이 1분위보다 더 높은 <교육>, <교통>, <음식·숙박> 물가는 평균 0.7% 상승하는 데 그쳤습니다.
TTS mp3 저장
3번째 뉴스입니다.사진 제공:연합뉴스이란과의 월드컵 최종예선을 앞둔 축구대표팀에 코로나19 비상이 걸렸습니다.
백승호는 대표팀 소집 첫날인 어제 15명의 선수 대상으로 실시한 신속항원검사에서 양성 반응을 보였고 이어진 PCR 검사에서도 확진 판정을 받았습니다.
대표팀은 해외파를 포함해 오늘 25명이 모두 모일 예정인 가운데, 경기 48시간 전인 오늘 저녁 전원 코로나19 검사를 실시할 예정입니다.
TTS mp3 저장


In [42]:
!pip install playsound

  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7037 sha256=f492506833b817cdb4b4a3a5ba83515f0f5551145edd0e1ef487056c53f00a9b
  Stored in directory: c:\users\jhjh3\appdata\local\pip\cache\wheels\ba\39\54\c8f7ff9a88a644d3c58b4dec802d90b79a2e0fb2a6b884bf82
Successfully built playsound


In [48]:
from playsound import playsound

In [70]:
playsound('./voice1.mp3')
playsound('./voice2.mp3')
playsound('./voice3.mp3')

In [71]:
audio1 = open("voice1.mp3", "rb").read()
audio2 = open("voice2.mp3", "rb").read()
audio3 = open("voice3.mp3", "rb").read()
audioJoin = audio1 + audio2 + audio3
audioFinal = open("audioFinal.mp3", "wb").write(audioJoin)

In [73]:
playsound('./audioFinal.mp3')